In [16]:
from policyengine_us.model_api import *

def create_eitc_winship_reform():
    # Compute EITC under filer_earned = tax_unit_head_earned
    # Then compute EITC under filer_earned = tax_unit_spouse_earned
    # Then set EITC = sum of the two

    class original_eitc(Variable):
        value_type = float
        entity = TaxUnit
        definition_period = YEAR
        label = "Original EITC"
        reference = "https://www.law.cornell.edu/uscode/text/26/32#a"
        unit = USD
        defined_for = "eitc_eligible"

        def formula(tax_unit, period, parameters):
            maximum = tax_unit("eitc_maximum", period)
            phased_in = tax_unit("eitc_phased_in", period)
            reduction = tax_unit("eitc_reduction", period)
            limitation = max_(0, maximum - reduction)
            return min_(phased_in, limitation)
        
    class earned_income_tax_credit(Variable):
        value_type = float
        entity = TaxUnit
        definition_period = YEAR
        label = "EITC"
        unit = USD
        defined_for = "eitc_eligible"

        def formula(tax_unit, period, parameters):
            person = tax_unit.members
            simulation = tax_unit.simulation
            agi = tax_unit("adjusted_gross_income", period)
            earned_income = person("earned", period)
            is_head = person("is_tax_unit_head", period)
            is_spouse = person("is_tax_unit_spouse", period)

            filer_earned_head_only = tax_unit.sum(earned_income * is_head)
            filer_earned_spouse_only = tax_unit.sum(earned_income * is_spouse)

            head_only_branch = simulation.get_branch("head_only")
            head_only_branch.set_input("filer_earned", period, filer_earned_head_only)
            head_only_branch.set_input("adjusted_gross_income", period, filer_earned_head_only)
            head_eitc = head_only_branch.calculate("original_eitc", period)

            spouse_only_branch = simulation.get_branch("spouse_only")
            spouse_only_branch.set_input("filer_earned", period, filer_earned_spouse_only)
            spouse_only_branch.set_input("adjusted_gross_income", period, filer_earned_spouse_only)
            spouse_eitc = spouse_only_branch.calculate("original_eitc", period)

            return (agi < 100_000) * (head_eitc + spouse_eitc)
    
    class winship_eitc_reform(Reform):
        def apply(self):
            self.add_variable(original_eitc)
            self.update_variable(earned_income_tax_credit)
    
    return winship_eitc_reform

In [17]:
from policyengine_us import Microsimulation

baseline = Microsimulation()
reformed = Microsimulation(reform=create_eitc_winship_reform())

reformed.calculate("household_net_income", 2023).sum() - baseline.calculate("household_net_income", 2023).sum()

39961936581.42383

In [18]:
(reformed.calculate("household_net_income", 2023).sum() - baseline.calculate("household_net_income", 2023).sum()) / 1e9

39.96193658142383

In [15]:
from policyengine_us import Simulation

simulation = Simulation(
    reform=create_eitc_winship_reform(),
    situation={
        "people": {
            "head": {
                "employment_income": 10_000
            },
            "spouse": {
                "employment_income": 30_000
            },
            "child": {
                "age": 5
            },
        },
        "tax_units": {
            "tax_unit": {
                "members": ["head", "spouse", "child"]
            }
        }
    }
)

simulation.trace = True
simulation.calculate("earned_income_tax_credit")
simulation.tracer.print_computation_log()

  earned_income_tax_credit<2022, (default)> = [6535.348]
    eitc_eligible<2022, (default)> = [ True]
      tax_unit_children<2022, (default)> = [1.]
        is_child<2022, (default)> = [False False  True]
          age<2022, (default)> = [40. 40.  5.]
      age<2022, (default)> = [40. 40.  5.]
      eitc_relevant_investment_income<2022, (default)> = [0.]
        capital_gains<2022, (default)> = [0. 0. 0.]
          short_term_capital_gains<2022, (default)> = [0. 0. 0.]
          long_term_capital_gains<2022, (default)> = [0. 0. 0.]
        net_investment_income<2022, (default)> = [0.]
          taxable_interest_income<2022, (default)> = [0. 0. 0.]
          dividend_income<2022, (default)> = [0. 0. 0.]
            qualified_dividend_income<2022, (default)> = [0. 0. 0.]
            non_qualified_dividend_income<2022, (default)> = [0. 0. 0.]
          rental_income<2022, (default)> = [0. 0. 0.]
          c01000<2022, (default)> = [0.]
            sep<2022, (default)> = [1]
            c